In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.cluster import KMeans
from sklearn.gaussian_process.kernels import Matern, RBF

import plotly.express as px

import plotly.graph_objects as go

from scipy.linalg import toeplitz, block_diag

from spe.mse_estimator import ErrorComparer
from spe.data_generation import gen_rbf_X, gen_matern_X, create_clus_split, gen_cov_mat
from spe.relaxed_lasso import RelaxedLasso
from spe.estimators import kfoldcv, kmeanscv, cp_relaxed_lasso_train_test, cp_adaptive_smoother_train_test, better_test_est_split

In [59]:
niter = 100

# n=30**2
# p=30
# s=30
n=20**2
p=200
s=30

k=10

delta = 0.8
length_scale = 1
nu = 2.5
lambd=.45

In [60]:
err_cmp = ErrorComparer()

In [61]:
nx = ny = int(np.sqrt(n))
xs = np.linspace(0, 10, nx)
ys = np.linspace(0, 10, ny)
c_x, c_y = np.meshgrid(xs, ys)
c_x = c_x.flatten()
c_y = c_y.flatten()
coord = np.stack([c_x, c_y]).T

In [62]:
Sigma_t = gen_cov_mat(c_x, c_y, RBF(length_scale=length_scale))
# Sigma_t = gen_cov_mat(c_x, c_y, Matern(length_scale=length_scale, nu=nu))
Sigma_t = delta*Sigma_t + (1-delta)*np.eye(n)

Chol_t = np.linalg.cholesky(Sigma_t)

# Sigma_t = np.eye(n)
# Chol_t = np.eye(n)

In [63]:
# c_x = np.random.uniform(0,10,size=n)
# c_y = np.random.uniform(0,10,size=n)

# rn = int(np.sqrt(n))
# ctr_x = np.random.uniform(0,10,size=rn)
# ctr_y = np.random.uniform(0,10,size=rn)
# c_x = np.concatenate([c + .35 * np.random.randn(rn) for c in ctr_x])
# c_y = np.concatenate([c + .35 * np.random.randn(rn) for c in ctr_y])

# X = gen_rbf_X(c_x, c_y, p)
X = gen_matern_X(c_x, c_y, p, length_scale=1, nu=1.5)
# X = np.random.randn(n,p)

beta = np.zeros(p)
idx = np.random.choice(p,size=s)
beta[idx] = np.random.uniform(-1,1,size=s)

In [64]:
# xv, yv = np.meshgrid(np.arange(nx), np.arange(ny))
# pts = np.stack([xv.ravel(), yv.ravel()]).T
# n = nx*ny
# rn = int(np.sqrt(n))
# ctr = np.random.choice(pts.shape[0], size=rn, replace=True)
# ctr = pts[ctr]
# tr_idx = np.vstack([[pt + np.array((1.25*np.random.randn(2)).astype(int)) for _ in np.arange(rn)] for pt in ctr])
# tr_idx = np.maximum(0, tr_idx)
# tr_idx[:,0] = cx = np.minimum(nx-1, tr_idx[:,0])
# tr_idx[:,1] = cy = np.minimum(ny-1, tr_idx[:,1])
# tr_idx = np.unique(np.ravel_multi_index(tr_idx.T, (nx,ny)))
# tr_idx.shape

In [65]:
(test_err,
 rnd_err,
 trc_err) = err_cmp.compare(RelaxedLasso(lambd=lambd),
#                                  [cp_relaxed_lasso_train_test, cp_relaxed_lasso_train_test],
#                                  [{'alpha':1., 'use_trace_corr': False}, {'alpha':1., 'use_trace_corr': True}],
                                 [better_test_est_split,
                                  cp_adaptive_smoother_train_test, 
                                  cp_adaptive_smoother_train_test],
                                 [{},
                                  {'alpha':1., 'use_trace_corr': False},#, 'full_refit':True}, 
                                  {'alpha':1., 'use_trace_corr': True}],#, 'full_refit':True}],
                                 niter=niter,
                                 n=n,
                                 p=p,
                                 s=p,
                                 snr=0.4, 
                                 X=X,
                                 beta=beta,
                                 coord=coord,
                                 Chol_t=Chol_t,
                                 Chol_s=None,
                                 tr_idx=None,
                                 fair=False,
                                 )

0
0.5625
10
20
30


/Users/kevinfry/Documents/GitHub/scikit-learn/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.767e+00, tolerance: 5.945e+00



40


/Users/kevinfry/Documents/GitHub/scikit-learn/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.393e+01, tolerance: 7.735e+00



50
60
70
80
90


In [66]:
# fig_X = go.Figure(data=[go.Mesh3d(x=c_x, y=c_y, z=X[:,13], opacity=0.50)])
# fig_X.update_layout(scene = dict(
#                     xaxis_title='Latitude',
#                     yaxis_title='Longitude',
#                     zaxis_title='Feature Xj'),
# )
# fig_X.show()

In [67]:
# fig_y = go.Figure(data=[go.Mesh3d(x=c_x, y=c_y, z=y, opacity=0.50)])
# fig_y.update_layout(scene = dict(
#                     xaxis_title='Latitude',
#                     yaxis_title='Longitude',
#                     zaxis_title='Response Y'),
# )
# fig_y.show()

In [68]:
# fig_mu = go.Figure(data=[go.Mesh3d(x=c_x, y=c_y, z=mu, opacity=0.50)])
# fig_mu.update_layout(scene = dict(
#                     xaxis_title='Latitude',
#                     yaxis_title='Longitude',
#                     zaxis_title='Mean'),
# )
# fig_mu.show()

In [69]:
# fig_eps = go.Figure(data=[go.Mesh3d(x=c_x, y=c_y, z=y-mu, opacity=0.50)])
# fig_eps.update_layout(scene = dict(
#                     xaxis_title='Latitude',
#                     yaxis_title='Longitude',
#                     zaxis_title='Eps'),
# )
# fig_eps.show()

In [70]:
# groups = KMeans(n_clusters=10).fit(tr_idx).labels_

In [71]:
# px.scatter(pd.DataFrame({'x': tr_idx[:,0], 'y': tr_idx[:,1], 'g': groups}), x='x', y='y', color='g')

In [72]:
risk = test_err.mean()
# risk_kfcv = kfcv_err.mean()
# risk_spcv = spcv_err.mean()
risk_rnd = rnd_err.mean()
risk_trc = trc_err.mean()
# risk, risk_kfcv, risk_spcv, risk_lin

In [73]:
df = pd.DataFrame({'Rand': (rnd_err.T),# - test_err.T) / test_err.T,
                   'Trace': (trc_err.T)})# - test_err.T) / test_err.T})

In [74]:
fig_lin = px.box((df - risk) / risk, 
             labels={
                     "variable": "Method",
                     "value": "Relative MSE"
                     },
             title="RL Correction",
             points=False)
fig_lin.update_traces(boxmean=True)

fig_lin.add_hline(y=0., line_color='red')

In [75]:
fig = go.Figure(data=go.Scatter(
        x=[0, 1, 2],
        y=((df - risk)/risk).mean(),
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=((df - risk)/risk).std(),
            visible=True)
    ))
fig.show()

In [76]:
(df/risk).mean()

Rand     0.840967
Trace    0.841799
dtype: float64

In [57]:
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Control',
    x=['Rand', 'Trace'], y=(df/risk).mean(),
    marker_color=px.colors.qualitative.Plotly,
    text=np.around((df/risk).mean(),3),
    textposition='outside',
    error_y=dict(
        type='data',
        color='black',
        symmetric=False,
        array=(df/risk).quantile(.75),
        arrayminus=(df/risk).quantile(.25))
))
# fig.add_trace(go.Bar(
#     name='Experimental',
#     x=['GenCp', 'KFCV', 'SPCV'], y=(df/risk).mean(),
#     error_y=dict(type='data', array=[1, 2])
# ))
# fig.update_layout(barmode='group')
fig.add_hline(y=1., line_color='red')
fig.update_layout(
    title="Linear Regression Test Error Estimates",
    xaxis_title="Method",
    yaxis_title="Relative MSE",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     )
)
fig.show()

In [77]:
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Control',
    x=['Rand', 'Trace'], y=(df/risk).mean(),
    marker_color=px.colors.qualitative.Plotly,
    text=np.around((df/risk).mean(),3),
    textposition='outside',
    error_y=dict(
        type='data',
        color='black',
        symmetric=False,
        array=(df/risk).quantile(.75),
        arrayminus=(df/risk).quantile(.25))
))
# fig.add_trace(go.Bar(
#     name='Experimental',
#     x=['GenCp', 'KFCV', 'SPCV'], y=(df/risk).mean(),
#     error_y=dict(type='data', array=[1, 2])
# ))
# fig.update_layout(barmode='group')
fig.add_hline(y=1., line_color='red')
fig.update_layout(
    title="Linear Regression Test Error Estimates",
    xaxis_title="Method",
    yaxis_title="Relative MSE",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     )
)
fig.show()

In [78]:
df.var()

Rand      85.751251
Trace    104.867962
dtype: float64